In [ ]:
# Import statements
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
import time
import math
import warnings

Get the options price data from [1]. \\
Alternatively, we have uploaded the data to Google Drive, and it can be downloaded from there [2]. \\
We do not need the full dataset, only the NIFTY and BANKNIFTY Call Options with Traded Value>1000 lakhs

In [ ]:
# Filter out the required data from the csv file
with open('3mfanddo.csv', 'r') as file:
    reader = csv.DictReader(file)   # Create a CSV reader object
    header = reader.fieldnames  # Define the header

    # Create an empty list to store filtered rows
    filtered_rows = []

    # Iterate over each row in the CSV file
    for row in reader:
        # Check if the 'INSTRUMENT' column has values 'OPTIDX'
        if (row['INSTRUMENT']=='OPTIDX') and (float(row['VAL_INLAKH'])>=1000) and (row['OPTION_TYP']=='CE'):
          filtered_rows.append(row)
data = pd.DataFrame(filtered_rows)
del filtered_rows   # To save RAM
print(data.shape)
print(data.columns)

# Find the range of dates in the dataframe
temp = pd.to_datetime(data['TIMESTAMP'], format='%d-%b-%Y')
min_date = temp.min()
max_date = temp.max()
print("Minimum date:", min_date)
print("Maximum date:", max_date)

FileNotFoundError: [Errno 2] No such file or directory: '3mfanddo.csv'

In [ ]:
# Remove the unnecessary columns and datapints
data = data.dropna(inplace=True)
data.drop(['', 'INSTRUMENT', 'OPTION_TYP', 'CLOSE', 'TIMESTAMP'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
# Get the underlying banknifty/nifty close data using yfinance api [4] [5]
!pip install yfinance
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

In [ ]:
# Get the underlying close price
# %%capture
import math
import warnings
warnings.simplefilter('ignore')

eta = 0
eta_count = 1
eta_time = 0
N = data.shape[0]

data['UNDER_CLOSE'] = 0   # Price of the underlying symbol

symbols_not_available = []
drop_indices = []
for index in range(N):
  row = data.loc[index]

  # To get the estimated time to completion
  if(eta_count == 1000 ):
    eta_count = 1
    eta = (time.time() - eta_time) * (N-index)/ 1000
    eta_time = time.time()
    print("ETA in seconds:", eta)
    print(index, '/', N)
  eta_count += 1

  # To check if under_close already exists
  if math.isnan(row['UNDER_CLOSE']) == False and row['UNDER_CLOSE']!=0:
    print(row)
    continue

  symbol = row['SYMBOL']
  date_str = row['TIMESTAMP']
  parsed_date = datetime.strptime(date_str, '%d-%b-%Y')
  next_date = parsed_date + timedelta(days=1)

  # These symbols have been delisted/discontinued
  if(symbol in symbols_not_available):
    data.drop(index=index, inplace=True)
    continue

  if(symbol=='NIFTY'):
    yf_symbol = '^NSEI'
  elif(symbol=='BANKNIFTY'):
    yf_symbol ='^NSEBANK'
  elif(symbol=='NIFTYIT'):
    yf_symbol = '^CNXIT'
  else:
    yf_symbol = symbol+".NS"

  try:
    close_data = yf.download(yf_symbol, start = parsed_date.strftime('%Y-%m-%d'), end = next_date.strftime('%Y-%m-%d'), progress=False)
  except:
    symbols_not_available.append(symbol)
    drop_indices.append(index)
    continue
  try:
    data.at[index, 'UNDER_CLOSE'] = close_data['Close'].values[0]
  except:
    symbols_not_available.append(symbol)
    drop_indices.append(index)
    continue

data.drop(drop_indices, axis=0, inplace=True)
print(data.columns)
print(data.shape)

In [ ]:
# Get the risk-free returns using the Government 10-Year Treasury Bill returns from [6]
# The csv file has been uploaded to drive at [3]
risk_data_path = 'India 10-Year Bond Yield Historical Data.csv'
risk_data = pd.read_csv(risk_data_path)
risk_data['Date'] = pd.to_datetime(risk_data['Date'], format='%d-%m-%Y')
print(risk_data.head())

In [ ]:
# Append the risk free interest rate to the dataframe
# Append the (expiry - date) (days to expiry) as a column in the dataframe, with time in years
# A year is taken as 252 days, excluding weekend holidays

data.reset_index(drop=True, inplace=True)
N = data.shape[0]
drop_rows = []
data['risk_free_rate'] = 0
for index in range(N):
  row = data.loc[index]
  date_str = row['TIMESTAMP']
  parsed_date = datetime.strptime(date_str, '%d-%b-%Y')
  expiry_str = row['EXPIRY_DT']
  parsed_expiry = datetime.strptime(expiry_str, '%d-%b-%Y')
  delta_days =parsed_expiry - parsed_date
  data.at[index,'to_expiry'] = delta_days/252

  temp_date = risk_data[risk_data['Date'] == parsed_date]
  if len(temp_date) == 0:
    drop_rows.append(index)
    continue
  risk_free_rate = temp_date['Price'].values[0]

  data.at[index,'risk_free_rate'] = risk_free_rate

data.drop(drop_rows, axis=0, inplace=True)
data.reset_index(drop=True, inplace=True)

data['to_expiry'] = data['to_expiry'].dt.days
data.drop(['EXPIRY_DT'], axis=1, inplace=True)

In [ ]:
# Calculate the volatilites (times 100), which is the annualised standard deviation

# Sort data according to TIMESTAMP column
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], format='%d-%b-%Y')
data = data.sort_values(by='TIMESTAMP')
data.reset_index(drop=True, inplace=True)

def update_history(history, price):
  if len(history) <= 10:
    history.append(price)
    return history
  for i in range(len(history)-1):
    history[i] = history[i+1]
  history[-1] = price
  return history

def get_annual_volatility(stock_prices):
  log_returns = [np.log(stock_prices[i] / stock_prices[i-1]) for i in range(1, len(stock_prices))]
  std_dev = np.std(log_returns)
  annualized_std_dev = std_dev * np.sqrt(252)
  return annualized_std_dev

# Get volatility
banknifty_volatilites = {}  # To store the dates for which vol has been calculated
nifty_volatilites = {}

banknifty_window = []   # TO get the rolling window to calculate vol
nifty_window = []

data['VOLATILITY'] = 0    # Initialise the column

for index in range(len(data)):
  row = data.loc[index]
  if row['SYMBOL'] == 'BANKNIFTY':
    if row['TIMESTAMP'] not in banknifty_volatilites:
      if len(banknifty_window) < 20:
        banknifty_window.append(row['UNDER_CLOSE'])
        banknifty_volatilites[row['TIMESTAMP']] = 0
      else:
        banknifty_window = update_history(banknifty_window, row['UNDER_CLOSE'])
        banknifty_volatilites[row['TIMESTAMP']] = get_annual_volatility(banknifty_window)
    data.at[index, 'VOLATILITY'] = banknifty_volatilites[row['TIMESTAMP']]
  else:
    if row['TIMESTAMP'] not in nifty_volatilites:
      if len(nifty_window) < 20:
        nifty_window.append(row['UNDER_CLOSE'])
        nifty_volatilites[row['TIMESTAMP']] = 0
      else:
        nifty_window = update_history(nifty_window, row['UNDER_CLOSE'])
        nifty_volatilites[row['TIMESTAMP']] = get_annual_volatility(nifty_window)
    data.at[index, 'VOLATILITY'] = nifty_volatilites[row['TIMESTAMP']]

data['VOLATILITY'] = data['VOLATILITY'] * 100
data = data[data['VOLATILITY'] != 0]    # Remove thosewith zero volatility

# Store the final dataset as a csv file for future purposes
data.to_csv('data_final.csv', index=False)
print(data.shape)

In [ ]:
# Get banknifty and nifty data for training the LSTM
banknifty_data = yf.download('^NSEBANK', start="2014-11-30", end="2019-11-30")
nifty_data = yf.download('^NSEI', start="2014-11-30", end="2019-11-30")

banknifty_data = banknifty_data[ 'Close']
nifty_data = nifty_data[ 'Close']

banknifty_data.to_csv('banknifty_data.csv')
nifty_data.to_csv('nifty_data.csv')

# *References*:
[1] https://www.kaggle.com/datasets/sunnysai12345/nse-future-and-options-dataset-3m
[2]https://drive.google.com/file/d/1LEfYIu2TYorRgzpMXc70FxslUtrkzg9G/view?usp=sharing   
[3] https://drive.google.com/file/d/17eWAPg10eQ3iEXcrbWtUwm0z8UFKhInj/view?usp=sharing  
[4] https://pypi.org/project/yfinance/    
[5] https://medium.com/@dhruvi31/initializing-yfinance-to-get-nse-stock-price-data-2d05b9c920f2     
[6] https://in.investing.com/rates-bonds/india-10-year-bond-yield-historical-data
